In [ ]:
#@markdown ## 구글드라이브 연동
#@markdown **생성한 이미지를 구글드라이브에 자동저장하려면 체크해주세요**
Google_Drive = False #@param {type:"boolean"}

#@markdown -----
#@markdown *아래는 설정할 필요 없어요*

#@markdown **구글드라이브에 연결 폴더**
Google_Drive_Dir = 'webui' #@param {type:"string"}

#@markdown **폴더 설명** : 미리 만드셔도 되고 없으면 자동으로 생성해요
#@markdown * webui/output : 생성된 이미지들이 저장되요
#@markdown * webui/checkpoint : 모델(checkpoint)를 넣어주면 읽어올수 있어요
#@markdown * webui/lora : 로라(LoRA)를 넣어주면 읽어올수 있어요
#@markdown * webui/embedding : 임베딩(Textual Inversion)를 넣어주면 읽어올수 있어요
#@markdown * webui/hyperwork : 하이퍼워크(Hyperworks)를 넣어주면 읽어올수 있어요
#@markdown * webui/wildcards : 와일드카드(Wildcards)를 넣어주면 읽어올수 있어요

#@markdown **초기모델(checkpoint) URL**
Checkpoint_Url = 'https://civitai.com/api/download/models/49055?type=Model&format=SafeTensor&size=pruned&fp=fp16' #@param {type:"string"}
#@markdown **초기모델 파일명**
Checkpoint_Filename = 'majicMIX_fantasy_v20.safetensors' #@param {type:"string"}

Workspace = 'animatediff'

if Google_Drive:
  from google.colab import drive
  drive.mount('/content/drive')

# https://github.com/camenduru 참고
# 메모리 Fix
!wget https://github.com/camenduru/gperftools/releases/download/v1.0/libtcmalloc_minimal.so.4 -O /content/libtcmalloc_minimal.so.4
%env LD_PRELOAD=/content/libtcmalloc_minimal.so.4

!apt -y install -qq aria2
!pip install xformers==0.0.20 triton==2.0.0 -U

%cd /content
!git clone https://github.com/neggles/animatediff-cli /content/{Workspace}
!gir reset --hard 877395691092c615d8f2f0c079bc7038578460c0

# 구글드라이브 연결
!wget https://raw.githubusercontent.com/ninjaneural/webui/master/misc/link_google_drive_animatediff.sh -O /content/link_google_drive.sh
!bash /content/link_google_drive.sh {Workspace} {Google_Drive} {Google_Drive_Dir}

%cd /content/{Workspace}
!pip install -e .

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{Checkpoint_Url}" -d /content/{Workspace}/data/models/sd -o {Checkpoint_Filename}
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v14.ckpt -d /content/{Workspace}/data/models/motion-module -o mm_sd_v14.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v15.ckpt -d /content/{Workspace}/data/models/motion-module -o mm_sd_v15.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/5637?type=Model&format=PickleTensor&size=full&fp=fp16" -d /content/{Workspace}/data/embeddings/ -o ng_deepnegative_v1_75t.pt

!wget https://raw.githubusercontent.com/ninjaneural/webui/master/misc/animatediff/cli.py -O /content/{Workspace}/src/animatediff/cli.py
!wget https://raw.githubusercontent.com/ninjaneural/webui/master/misc/animatediff/prompt.json -O /content/{Workspace}/config/prompts/prompt.json

!sed -i -e "s/#Checkpoint_Filename#/{Checkpoint_Filename}/" /content/{Workspace}/config/prompts/prompt.json


In [ ]:
!animatediff generate -c 'config/prompts/prompt.json' -W 512 -H 768 -L 16